In [1]:
import sqlite3

In [2]:
import pandas as pd

In [3]:
conn = sqlite3.connect("db/pharm_data.db", check_same_thread=False)

In [19]:
global_vars={}
global_vars['prc'] = '14'
global_vars['meds'] = map(str,[197319,
1049221,
835603,
313988,
859751,
896185])
user_zip = '43201'

out = pd.read_sql("select \
                PLAN_INFO.CONTRACT_NAME, \
                PLAN_INFO.PLAN_NAME, \
                PLAN_INFO.CONTRACT_ID, \
                PLAN_INFO.PLAN_ID, \
                PLAN_INFO.SEGMENT_ID, \
                PLAN_INFO.FORMULARY_ID, \
                PLAN_INFO.PREMIUM, \
                PLAN_INFO.DEDUCTIBLE \
        from \
            FORMULARY_DATA join PLAN_INFO \
            on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
        where \
              PLAN_INFO.PDP_REGION_CODE is '"+str(global_vars['prc'])+"' and \
              PLAN_INFO.CONTRACT_ID like 'S%'\
        order by PLAN_INFO.FORMULARY_ID",conn)\
.drop_duplicates(subset='FORMULARY_ID',keep='first')\
.reset_index(drop=True)
for med in global_vars['meds']:
        out = pd.concat([out,
           pd.read_sql(" ".join(("select \
                    PLAN_INFO.CONTRACT_NAME, \
                    PLAN_INFO.PLAN_NAME, \
                    PLAN_INFO.CONTRACT_ID, \
                    PLAN_INFO.PLAN_ID, \
                    PLAN_INFO.SEGMENT_ID, \
                    PLAN_INFO.FORMULARY_ID, \
                    PLAN_INFO.PREMIUM, \
                    PLAN_INFO.DEDUCTIBLE, \
                    FORMULARY_DATA.RXCUI as MED_"+med+", \
                    FORMULARY_DATA.PRIOR_AUTHORIZATION_YN as PA_"+med+", \
                    FORMULARY_DATA.TIER_LEVEL_VALUE as TIER_"+med+", \
                    BENEFICIARY_COST.COST_AMT_PREF as CAP_"+med+" \
            from \
                FORMULARY_DATA join PLAN_INFO \
                    on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
                join BENEFICIARY_COST \
                    on PLAN_INFO.CONTRACT_ID = BENEFICIARY_COST.CONTRACT_ID and \
                       PLAN_INFO.PLAN_ID = BENEFICIARY_COST.PLAN_ID and \
                       cast(FORMULARY_DATA.TIER_LEVEL_VALUE as integer) = \
                                                cast(BENEFICIARY_COST.TIER as integer) \
            where \
                  PLAN_INFO.PDP_REGION_CODE is '"+str(global_vars['prc'])+"' and \
                  FORMULARY_DATA.RXCUI like '%"+med+"' and \
                  PLAN_INFO.CONTRACT_ID like 'S%' and \
                  (cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 0 or \
                  cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 1) and \
                  BENEFICIARY_COST.DAYS_SUPPLY = 1 and \
                  cast(BENEFICIARY_COST.COST_TYPE_PREF as integer) = 1\
            order by PLAN_INFO.FORMULARY_ID").split()),conn)\
        .drop_duplicates(subset='FORMULARY_ID',keep='first')\
        .reset_index(drop=True)\
        [['MED_'+med,
          'PA_'+med,
          'TIER_'+med,
          'CAP_'+med]]],axis=1)
#ADD IN COST COLUMN
        if 'ANNUAL_TOTAL_COST' in out.columns:
            del out['ANNUAL_TOTAL_COST']
        if 'MONTHLY_COPAY' in out.columns:
            del out['MONTHLY_COPAY']
        out['ANNUAL_TOTAL_COST'] = (out[[c for c in out.columns if c[:3] == 'CAP']]*12).sum(axis=1) + out['PREMIUM']*12 + out['DEDUCTIBLE']
        out['MONTHLY_COPAY'] = out[[c for c in out.columns if c[:3] == 'CAP']].sum(axis=1)
        out = out.sort_values(by='ANNUAL_TOTAL_COST').reset_index(drop=True).dropna()

display(out)

,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,MED_197319,PA_197319,...,MED_859751,PA_859751,TIER_859751,CAP_859751,MED_896185,PA_896185,TIER_896185,CAP_896185,ANNUAL_TOTAL_COST,MONTHLY_COPAY
0,MEDCO CONTAINMENT LIFE AND MEDCO CONTAINMENT NY,Express Scripts Medicare - Saver (PDP),S5660,230.0,0.0,18152,22.6,405.0,197319,N,...,859751,N,2,12.0,896185,N,3.0,30.0,1384.2,59.0
1,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,20.0,0.0,18041,25.7,405.0,197319,N,...,859751,N,2,3.0,896185,N,3.0,37.0,1397.4,57.0
2,HUMANA INSURANCE COMPANY,Humana Walmart Rx Plan (PDP),S5884,160.0,0.0,18255,18.4,405.0,197319,N,...,859751,N,2,5.0,896185,N,3.0,35.0,1561.8,78.0
3,SILVERSCRIPT INSURANCE COMPANY,SilverScript Choice (PDP),S5601,28.0,0.0,18419,24.0,0.0,197319,N,...,859751,N,2,6.0,896185,N,3.0,47.0,1608.0,110.0
4,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,137.0,0.0,18059,56.2,0.0,197319,N,...,859751,N,2,14.0,896185,N,3.0,47.0,1646.4,81.0
5,SILVERSCRIPT INSURANCE COMPANY,SilverScript Plus (PDP),S5601,29.0,0.0,18420,46.3,0.0,197319,N,...,859751,N,2,10.0,896185,N,3.0,42.0,1743.6,99.0
6,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,48.0,0.0,18056,30.7,350.0,197319,N,...,859751,N,3,25.0,896185,N,3.0,28.0,1798.4,90.0
8,HUMANA INSURANCE COMPANY,Humana Preferred Rx Plan (PDP),S5884,137.0,0.0,18254,31.1,405.0,197319,N,...,859751,N,2,1.0,896185,N,3.0,42.0,1858.2,90.0
10,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,395.0,0.0,18043,28.5,405.0,197319,N,...,859751,N,3,28.0,896185,N,3.0,29.0,1935.0,99.0
11,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,288.0,0.0,18057,16.8,405.0,197319,N,...,859751,N,3,47.0,896185,N,3.0,35.0,1986.6,115.0


In [ ]:
out[:5]

In [17]:
i=0
contract_id = out[i:i+1]["CONTRACT_ID"].values[0]
plan_id = str(int(out[i:i+1]["PLAN_ID"].values[0]))
#     display(contract_id)
#     display(plan_id)
df = pd.read_sql("select NCPDP.PHARMACY_NAME, NCPDP.PHONE, PNF_DATA.* \
from PNF_DATA, NCPDP \
where PNF_DATA.CONTRACT_ID is '"+contract_id+"' \
 and PNF_DATA.PLAN_ID is '"+"0"*(3-len(plan_id))+str(plan_id)+"' and\
    PNF_DATA.PHARMACY_ZIPCODE is '"+user_zip+"' and \
    NCPDP.NPI is substr(PNF_DATA.PHARMACY_NUMBER,3) limit 10",conn)
display(df)
    

,PHARMACY_NAME,PHONE,CONTRACT_ID,PLAN_ID,SEGMENT_ID,PHARMACY_NUMBER,PHARMACY_ZIPCODE,PREFERRED_STATUS_RETAIL,PREFERRED_STATUS_MAIL,PHARMACY_RETAIL,PHARMACY_MAIL,IN_AREA_FLAG,BRAND_DISPENSING_FEE_30,BRAND_DISPENSING_FEE_60,BRAND_DISPENSING_FEE_90,GENERIC_DISPENSING_FEE_30,GENERIC_DISPENSING_FEE_60,GENERIC_DISPENSING_FEE_90
0,KROGER PHARMACY,6142941696,S5660,230,000,101528092327,43201,Y,N,Y,N,1,0.4,0,0,0.6,0,0
1,CVS PHARMACY,6142942442,S5660,230,000,101386747129,43201,Y,N,Y,N,1,0.45,0,0,0.5,0,0
2,GENOA HEALTHCARE,6142978766,S5660,230,000,101750534772,43201,N,N,Y,N,1,2.5,0,0,2.5,0,0
3,WILDERS CITY PHARMACY,6147325947,S5660,230,000,101013253806,43201,N,N,Y,N,1,0.25,0,0,0.25,0,0
4,AMC OHIO PHARMACY,6143406776,S5660,230,000,101114287547,43201,N,N,Y,N,1,0.25,0,0,0.25,0,0
5,PLANNED PARENTHOOD OF GREATER OHIO CO,6142223604,S5660,230,000,101336278860,43201,N,N,Y,N,1,1.5,0,0,1.5,0,0


In [ ]:
pd.read_sql("select * from NCPDP join PNF_DATA on NCPDP.NPI = substr(PNF_DATA.PHARMACY_NUMBER,3) limit 10",conn)

In [ ]:
pd.read_sql("select * from NCPDP limit 10",conn)

In [ ]:
pd.read_sql("select * from PNF_DATA limit 20",conn)

In [27]:
pd.set_option('display.max_colwidth',-1)
out['SELECT'] = out[['CONTRACT_ID','PLAN_ID']].apply(lambda x: "<input type=\"checkbox\" value=\"{0},{1}\" name=\"pick_plan\">".format(x['CONTRACT_ID'],"0"*(3-len(str(int(x['PLAN_ID']))))+str(int(x['PLAN_ID']))))
out

KeyError: ('CONTRACT_NAME', 'occurred at index CONTRACT_NAME')

In [52]:
out['SELECT'] = out.apply(lambda row: "<input type=\"checkbox\" value=\""+row['CONTRACT_ID']+","+"0"*(3-len(str(int(row['PLAN_ID']))))+str(int(row['PLAN_ID']))+"\" class=\"pick_plan\">",axis=1)
out

,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,MED_197319,PA_197319,...,PA_859751,TIER_859751,CAP_859751,MED_896185,PA_896185,TIER_896185,CAP_896185,ANNUAL_TOTAL_COST,MONTHLY_COPAY,SELECT
0,MEDCO CONTAINMENT LIFE AND MEDCO CONTAINMENT NY,Express Scripts Medicare - Saver (PDP),S5660,230.0,0.0,18152,22.6,405.0,197319,N,...,N,2,12.0,896185,N,3.0,30.0,1384.2,59.0,"<input type=""checkbox"" value=""S5660,230"" class=""pick_plan"">"
1,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,20.0,0.0,18041,25.7,405.0,197319,N,...,N,2,3.0,896185,N,3.0,37.0,1397.4,57.0,"<input type=""checkbox"" value=""S0522,020"" class=""pick_plan"">"
2,HUMANA INSURANCE COMPANY,Humana Walmart Rx Plan (PDP),S5884,160.0,0.0,18255,18.4,405.0,197319,N,...,N,2,5.0,896185,N,3.0,35.0,1561.8,78.0,"<input type=""checkbox"" value=""S5884,160"" class=""pick_plan"">"
3,SILVERSCRIPT INSURANCE COMPANY,SilverScript Choice (PDP),S5601,28.0,0.0,18419,24.0,0.0,197319,N,...,N,2,6.0,896185,N,3.0,47.0,1608.0,110.0,"<input type=""checkbox"" value=""S5601,028"" class=""pick_plan"">"
4,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,137.0,0.0,18059,56.2,0.0,197319,N,...,N,2,14.0,896185,N,3.0,47.0,1646.4,81.0,"<input type=""checkbox"" value=""S5768,137"" class=""pick_plan"">"
5,SILVERSCRIPT INSURANCE COMPANY,SilverScript Plus (PDP),S5601,29.0,0.0,18420,46.3,0.0,197319,N,...,N,2,10.0,896185,N,3.0,42.0,1743.6,99.0,"<input type=""checkbox"" value=""S5601,029"" class=""pick_plan"">"
6,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,48.0,0.0,18056,30.7,350.0,197319,N,...,N,3,25.0,896185,N,3.0,28.0,1798.4,90.0,"<input type=""checkbox"" value=""S5810,048"" class=""pick_plan"">"
8,HUMANA INSURANCE COMPANY,Humana Preferred Rx Plan (PDP),S5884,137.0,0.0,18254,31.1,405.0,197319,N,...,N,2,1.0,896185,N,3.0,42.0,1858.2,90.0,"<input type=""checkbox"" value=""S5884,137"" class=""pick_plan"">"
10,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,395.0,0.0,18043,28.5,405.0,197319,N,...,N,3,28.0,896185,N,3.0,29.0,1935.0,99.0,"<input type=""checkbox"" value=""S5921,395"" class=""pick_plan"">"
11,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,288.0,0.0,18057,16.8,405.0,197319,N,...,N,3,47.0,896185,N,3.0,35.0,1986.6,115.0,"<input type=""checkbox"" value=""S5810,288"" class=""pick_plan"">"
